In [ ]:
%%writefile tree_packer_v21.cpp
// Tree Packer v21 - ENHANCED v19 with SWAP MOVES + MULTI-START
// All n values (1-200) processed in parallel + aggressive exploration
// NEW: Swap move operator, multi-angle restarts, higher temperature SA
// Compile: g++ -O3 -march=native -std=c++17 -fopenmp -o tree_packer_v21 tree_packer_v21.cpp

#include <bits/stdc++.h>
#include <omp.h>
using namespace std;

constexpr int MAX_N = 200;
constexpr int NV = 15;
constexpr double PI = 3.14159265358979323846;

alignas(64) const long double TX[NV] = {0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125};
alignas(64) const long double TY[NV] = {0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5};

struct FastRNG {
    uint64_t s[2];
    FastRNG(uint64_t seed = 42) {
        s[0] = seed ^ 0x853c49e6748fea9bULL;
        s[1] = (seed * 0x9e3779b97f4a7c15ULL) ^ 0xc4ceb9fe1a85ec53ULL;
    }
    inline uint64_t rotl(uint64_t x, int k) { return (x << k) | (x >> (64 - k)); }
    inline uint64_t next() {
        uint64_t s0 = s[0], s1 = s[1], r = s0 + s1;
        s1 ^= s0; s[0] = rotl(s0, 24) ^ s1 ^ (s1 << 16); s[1] = rotl(s1, 37);
        return r;
    }
    inline long double rf() { return (next() >> 11) * 0x1.0p-53L; }
    inline long double rf2() { return rf() * 2.0L - 1.0L; }
    inline int ri(int n) { return next() % n; }
    inline long double gaussian() {
        long double u1 = rf() + 1e-10L, u2 = rf();
        return sqrtl(-2.0L * logl(u1)) * cosl(2.0L * PI * u2);
    }
};

struct Poly {
    long double px[NV], py[NV];
    long double x0, y0, x1, y1;
};

inline void getPoly(long double cx, long double cy, long double deg, Poly& q) {
    long double rad = deg * (PI / 180.0L);
    long double s = sinl(rad), c = cosl(rad);
    long double minx = 1e9L, miny = 1e9L, maxx = -1e9L, maxy = -1e9L;
    for (int i = 0; i < NV; i++) {
        long double x = TX[i] * c - TY[i] * s + cx;
        long double y = TX[i] * s + TY[i] * c + cy;
        q.px[i] = x; q.py[i] = y;
        if (x < minx) minx = x; if (x > maxx) maxx = x;
        if (y < miny) miny = y; if (y > maxy) maxy = y;
    }
    q.x0 = minx; q.y0 = miny; q.x1 = maxx; q.y1 = maxy;
}

inline bool pip(long double px, long double py, const Poly& q) {
    bool in = false;
    int j = NV - 1;
    for (int i = 0; i < NV; i++) {
        if ((q.py[i] > py) != (q.py[j] > py) &&
            px < (q.px[j] - q.px[i]) * (py - q.py[i]) / (q.py[j] - q.py[i]) + q.px[i])
            in = !in;
        j = i;
    }
    return in;
}

inline bool segInt(long double ax, long double ay, long double bx, long double by,
                   long double cx, long double cy, long double dx, long double dy) {
    long double d1 = (dx-cx)*(ay-cy) - (dy-cy)*(ax-cx);
    long double d2 = (dx-cx)*(by-cy) - (dy-cy)*(bx-cx);
    long double d3 = (bx-ax)*(cy-ay) - (by-ay)*(cx-ax);
    long double d4 = (bx-ax)*(dy-ay) - (by-ay)*(dx-ax);
    return ((d1 > 0) != (d2 > 0)) && ((d3 > 0) != (d4 > 0));
}

inline bool overlap(const Poly& a, const Poly& b) {
    if (a.x1 < b.x0 || b.x1 < a.x0 || a.y1 < b.y0 || b.y1 < a.y0) return false;
    for (int i = 0; i < NV; i++) {
        if (pip(a.px[i], a.py[i], b)) return true;
        if (pip(b.px[i], b.py[i], a)) return true;
    }
    for (int i = 0; i < NV; i++) {
        int ni = (i + 1) % NV;
        for (int j = 0; j < NV; j++) {
            int nj = (j + 1) % NV;
            if (segInt(a.px[i], a.py[i], a.px[ni], a.py[ni],
                      b.px[j], b.py[j], b.px[nj], b.py[nj])) return true;
        }
    }
    return false;
}

struct Cfg {
    int n;
    long double x[MAX_N], y[MAX_N], a[MAX_N];
    Poly pl[MAX_N];
    long double gx0, gy0, gx1, gy1;

    inline void upd(int i) { getPoly(x[i], y[i], a[i], pl[i]); }
    inline void updAll() { for (int i = 0; i < n; i++) upd(i); updGlobal(); }

    inline void updGlobal() {
        gx0 = gy0 = 1e9L; gx1 = gy1 = -1e9L;
        for (int i = 0; i < n; i++) {
            if (pl[i].x0 < gx0) gx0 = pl[i].x0;
            if (pl[i].x1 > gx1) gx1 = pl[i].x1;
            if (pl[i].y0 < gy0) gy0 = pl[i].y0;
            if (pl[i].y1 > gy1) gy1 = pl[i].y1;
        }
    }

    inline bool hasOvl(int i) const {
        for (int j = 0; j < n; j++)
            if (i != j && overlap(pl[i], pl[j])) return true;
        return false;
    }

    inline bool anyOvl() const {
        for (int i = 0; i < n; i++)
            for (int j = i + 1; j < n; j++)
                if (overlap(pl[i], pl[j])) return true;
        return false;
    }

    inline long double side() const { return max(gx1 - gx0, gy1 - gy0); }
    inline long double score() const { long double s = side(); return s * s / n; }
};

// Squeeze
Cfg squeeze(Cfg c) {
    long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
    for (long double scale = 0.9995L; scale >= 0.98L; scale -= 0.0005L) {
        Cfg trial = c;
        for (int i = 0; i < c.n; i++) {
            trial.x[i] = cx + (c.x[i] - cx) * scale;
            trial.y[i] = cy + (c.y[i] - cy) * scale;
        }
        trial.updAll();
        if (!trial.anyOvl()) c = trial;
        else break;
    }
    return c;
}

// Compaction
Cfg compaction(Cfg c, int iters) {
    long double bs = c.side();
    for (int it = 0; it < iters; it++) {
        long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
        bool improved = false;
        for (int i = 0; i < c.n; i++) {
            long double ox = c.x[i], oy = c.y[i];
            long double dx = cx - c.x[i], dy = cy - c.y[i];
            long double d = sqrtl(dx*dx + dy*dy);
            if (d < 1e-6L) continue;
            for (long double step : {0.02L, 0.008L, 0.003L, 0.001L, 0.0004L}) {
                c.x[i] = ox + dx/d * step; c.y[i] = oy + dy/d * step; c.upd(i);
                if (!c.hasOvl(i)) {
                    c.updGlobal();
                    if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; ox = c.x[i]; oy = c.y[i]; }
                    else { c.x[i] = ox; c.y[i] = oy; c.upd(i); }
                } else { c.x[i] = ox; c.y[i] = oy; c.upd(i); }
            }
        }
        c.updGlobal();
        if (!improved) break;
    }
    return c;
}

// Local search
Cfg localSearch(Cfg c, int maxIter) {
    long double bs = c.side();
    const long double steps[] = {0.01L, 0.004L, 0.0015L, 0.0006L, 0.00025L, 0.0001L};
    const long double rots[] = {5.0L, 2.0L, 0.8L, 0.3L, 0.1L};
    const int dx[] = {1,-1,0,0,1,1,-1,-1};
    const int dy[] = {0,0,1,-1,1,-1,1,-1};

    for (int iter = 0; iter < maxIter; iter++) {
        bool improved = false;
        for (int i = 0; i < c.n; i++) {
            long double cx = (c.gx0 + c.gx1) / 2.0L, cy = (c.gy0 + c.gy1) / 2.0L;
            long double ddx = cx - c.x[i], ddy = cy - c.y[i];
            long double dist = sqrtl(ddx*ddx + ddy*ddy);
            if (dist > 1e-6L) {
                for (long double st : steps) {
                    long double ox = c.x[i], oy = c.y[i];
                    c.x[i] += ddx/dist * st; c.y[i] += ddy/dist * st; c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; }
                        else { c.x[i]=ox; c.y[i]=oy; c.upd(i); c.updGlobal(); } }
                    else { c.x[i]=ox; c.y[i]=oy; c.upd(i); }
                }
            }
            for (long double st : steps) {
                for (int d = 0; d < 8; d++) {
                    long double ox=c.x[i], oy=c.y[i];
                    c.x[i] += dx[d]*st; c.y[i] += dy[d]*st; c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; }
                        else { c.x[i]=ox; c.y[i]=oy; c.upd(i); c.updGlobal(); } }
                    else { c.x[i]=ox; c.y[i]=oy; c.upd(i); }
                }
            }
            for (long double rt : rots) {
                for (long double da : {rt, -rt}) {
                    long double oa = c.a[i]; c.a[i] += da;
                    while (c.a[i] >= 360.0L) c.a[i] -= 360.0L;
                    while (c.a[i] < 0.0L) c.a[i] += 360.0L;
                    c.upd(i);
                    if (!c.hasOvl(i)) { c.updGlobal(); if (c.side() < bs - 1e-12L) { bs = c.side(); improved = true; }
                        else { c.a[i]=oa; c.upd(i); c.updGlobal(); } }
                    else { c.a[i]=oa; c.upd(i); }
                }
            }
        }
        if (!improved) break;
    }
    return c;
}

// Simulated Annealing
Cfg simulatedAnnealing(Cfg c, int max_iter, double start_temp, double end_temp, FastRNG& rng) {
    long double current_score = c.side();
    Cfg best_c = c;
    long double best_score = current_score;
    
    for (int i = 0; i < max_iter; i++) {
        double temp = start_temp * pow(end_temp / start_temp, (double)i / max_iter);
        Cfg next_c = c;
        
        // Move type: 0=shift, 1=rotate, 2=swap
        int move_type = rng.ri(3);
        int idx = rng.ri(c.n);
        
        if (move_type == 0) {
            next_c.x[idx] += rng.rf2() * temp;
            next_c.y[idx] += rng.rf2() * temp;
        } else if (move_type == 1) {
            next_c.a[idx] += rng.rf2() * temp * 10.0;
        } else {
            int idx2 = rng.ri(c.n);
            if (idx != idx2) {
                swap(next_c.x[idx], next_c.x[idx2]);
                swap(next_c.y[idx], next_c.y[idx2]);
                swap(next_c.a[idx], next_c.a[idx2]);
            }
        }
        
        next_c.updAll();
        if (!next_c.anyOvl()) {
            long double next_score = next_c.side();
            if (next_score < current_score || rng.rf() < exp((current_score - next_score) / temp)) {
                c = next_c;
                current_score = next_score;
                if (current_score < best_score) {
                    best_score = current_score;
                    best_c = c;
                }
            }
        }
    }
    return best_c;
}

// Global best tracking
Cfg best_configs[MAX_N + 1];
long double best_scores[MAX_N + 1];

void load_best() {
    ifstream f("submission.csv");
    if (!f.is_open()) return;
    string line;
    getline(f, line); // header
    map<int, vector<tuple<long double, long double, long double>>> data;
    while (getline(f, line)) {
        stringstream ss(line);
        string segment;
        vector<string> parts;
        while (getline(ss, segment, ',')) parts.push_back(segment);
        if (parts.size() < 4) continue;
        
        string id_str = parts[0];
        size_t u1 = id_str.find('_');
        int grp = stoi(id_str.substr(0, u1));
        
        long double x = stold(parts[1]);
        long double y = stold(parts[2]);
        long double a = stold(parts[3]);
        data[grp].emplace_back(x, y, a);
    }
    
    for (auto& [n, vec] : data) {
        if (n > MAX_N) continue;
        Cfg c; c.n = n;
        for (int i = 0; i < n; i++) {
            auto [x, y, a] = vec[i];
            c.x[i] = x; c.y[i] = y; c.a[i] = a;
        }
        c.updAll();
        best_configs[n] = c;
        best_scores[n] = c.score();
    }
}

void save_best() {
    ofstream f("submission_v21.csv");
    f << "id,x,y,deg\n";
    f << fixed << setprecision(15);
    for (int n = 1; n <= MAX_N; n++) {
        if (best_configs[n].n == 0) continue;
        for (int i = 0; i < n; i++) {
            f << n << "_" << i << "," << best_configs[n].x[i] << "," << best_configs[n].y[i] << "," << best_configs[n].a[i] << "\n";
        }
    }
}

int main(int argc, char** argv) {
    int n_iter = 1000;
    int r_seed = 42;
    if (argc > 1) n_iter = atoi(argv[1]);
    if (argc > 2) r_seed = atoi(argv[2]);
    
    // Initialize best scores
    for (int i = 0; i <= MAX_N; i++) best_scores[i] = 1e18L;
    
    load_best();
    
    cout << "Starting optimization with " << n_iter << " iterations per N..." << endl;
    
    #pragma omp parallel for schedule(dynamic, 1)
    for (int n = 1; n <= MAX_N; n++) {
        FastRNG rng(r_seed + n * 1000 + omp_get_thread_num());
        
        // If we don't have a valid config, initialize random
        if (best_configs[n].n == 0) {
            Cfg c; c.n = n;
            int side = (int)ceil(sqrt(n));
            for(int i=0; i<n; i++) {
                c.x[i] = (i % side) * 1.5;
                c.y[i] = (i / side) * 1.5;
                c.a[i] = 0;
            }
            c.updAll();
            best_configs[n] = c;
            best_scores[n] = c.score();
        }
        
        Cfg current = best_configs[n];
        
        // Optimization loop
        for (int iter = 0; iter < n_iter; iter++) {
            // Perturb
            Cfg candidate = current;
            
            // Apply SA
            candidate = simulatedAnnealing(candidate, 1000, 0.5, 0.001, rng);
            
            // Squeeze
            candidate = squeeze(candidate);
            
            // Local Search
            candidate = localSearch(candidate, 50);
            
            // Compaction
            candidate = compaction(candidate, 20);
            
            long double score = candidate.score();
            
            #pragma omp critical
            {
                if (score < best_scores[n]) {
                    best_scores[n] = score;
                    best_configs[n] = candidate;
                    cout << "New best for N=" << n << ": " << score << endl;
                }
            }
            
            // Accept if close (to maintain diversity in local thread)
            if (score < current.score() * 1.05) {
                current = candidate;
            } else {
                // Reset to best occasionally
                if (rng.rf() < 0.1) current = best_configs[n];
            }
        }
    }
    
    save_best();
    
    // Calculate total score
    long double total_score = 0;
    for (int n = 1; n <= MAX_N; n++) total_score += best_scores[n];
    cout << "Final Total Score: " << total_score << endl;
    
    return 0;
}


In [ ]:
!g++ -O3 -march=native -std=c++17 -fopenmp -o tree_packer_v21 tree_packer_v21.cpp

In [ ]:
!./tree_packer_v21 100 42